# Background
Hello everyone, today I am learning about Keras (and Tensorflow) with the help of Jason Brownlee 
http://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

Tensorflow is a framework for building, training, and computing deep learning models. But Tensorflow is not alone as Theano is a popular alternative. Keras helps us abstract a bit from these two frameworks.

## Example
In the example below, we use a publically available dataset. From the tutorial, "It describes patient medical record data for Pima Indians and whether they had an onset of diabetes within five years."

## Imports & Building the Model

We begin by importing Keras and Numpy libraries and initializing a random seed for repeatability.

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

Using TensorFlow backend.


I diverged a bit here to download the dataset. The tutorial references it but this can improve repeatability.

In [2]:
from urllib.request import urlopen
html = urlopen("http://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data")
file = open("pima-indians-diabetes.csv", 'wb')
file.write(html.read())

23279

Back to Jason's code. Here we open the dataset file and split the input from the output.

* For X, we select the first 8 columns for all rows
* For Y, we select the final column for all rows

In [3]:
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

Afterward, we build the model. In Keras, the model is of type Sequential. We are going to build a network with a single hidden layer.

In [4]:
# create model
model = Sequential()

We can add successive layers. Let's add an input layer.

We do this by calling .add() on our Sequential model object. We pass a Dense() object to represent that the nodes are fully connected. Remember back to Andrew Ng's machine learning course? This means all the inputs are connected to all of the first nodes of the graph.

The Dense() constructor has a few arguments of its own specified here https://keras.io/layers/core/. 
* The first argument is the output dimension of 12 meaning our 8 inputs will be initially sent to 12 nodes each with its own output.
* The second parameter (which is named) is the input dimension which is equal to 8, which is the number of columns in our dataset not counting the final label column.
* The third argument is the init parameter which determines how the weights of this layer are initialized. In this case a uniform random distribution is specified. Specifying this type of distribution will actually help the model break symmetry. A concise explanation for why can be found on StackOverflow and a detailed explanation is also available in the machine learning course. http://stackoverflow.com/questions/20027598/why-should-weights-of-neural-networks-be-initialized-to-random-numbers
* The last parameter named activation specifies the activation type of the node. In this case, we are using relu or rectified linear unit. This is different from the logistic sigmoid that is common in introductory courses and is employed with convolutional nets (CNNs). Among others, the advantages can be faster convergence and eliminating vanishing gradients (your learning rate drops to 0). There are a few specific types of relu's too: https://en.wikipedia.org/wiki/Rectifier_(neural_networks)

Here's the code:

In [5]:
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))

Next, we need a hidden layer. Notice how Keras doesn't require us to specify the input_dim of this layer.

In [6]:
model.add(Dense(8, init='uniform', activation='relu'))

Finally, we add our output layer which has a single sigmoid output. The sigmoid function is bounded between 0 and 1, unlike the relu activation function. This is to make it easier on the authors to determine the class the network outputs.

In [7]:
model.add(Dense(1, init='uniform', activation='sigmoid'))

## Compiling the model

After we describe the architecture in Keras, we need to compile it down to a backend (TensorFlow or Theano). These libraries will then determine the most efficent way to run the network.

To compile, we simply call .compile() on the model. Here is the reference: https://keras.io/models/sequential/

We pass a few arguments to compile():
* loss is the loss function used for evaluating your model performance during training. This is also a bit different from introductory courses that focus on classification error or mean squared error. An informal empirical comparison is done in this article https://jamesmccaffrey.wordpress.com/2013/11/05/why-you-should-use-cross-entropy-error-instead-of-classification-error-or-mean-squared-error-for-neural-network-classifier-training/
* optimizer is the our optimization function. Here 'adam' refers to an efficient gradient descent implementation. I think the paper linked from the article is worth linking again: https://arxiv.org/pdf/1412.6980.pdf
* The last parameter is the metrics parameter which is a "list of metrics to be evaluated by the model during training and testing"

In [8]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fitting the model

We are getting close to results now! Of course, fitting the model is an easy call too. We just need to call .fit() with a few reasonable parameters:
* The first parameter is the input or in our case, X
* The second parameter is the label array or in our case, Y
* The third parameter is nb_epoch or number of epochs. Epoch is a fancy term for how many times we iterate over the training set and should not be confused with batch or online learning methods: http://stackoverflow.com/questions/31155388/meaning-of-an-epoch-in-neural-networks-training
* The fourth parameter is the batch_size which we set to 10. This effectively means we are using mini-batch learning. The advantage to mini-batch learning is that we can converge faster than online learning while not consuming huge amounts of resources computing over a full batch.
* Lastly, since this is a Jupyter notebook, we want to disable the output. This looks great on a command line but not so great here.

The next step will take a bit of time:

In [9]:
# Fit the model
model.fit(X, Y, nb_epoch=150, batch_size=10, verbose=0)

## Evaluation

So how did we do? Of course it's easy to find out.

Just call .evaluate() on your model and pass the input and the labels.

In [10]:
# evaluate the model
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 78.26%


So how did we do? Not very good by most industry or medical standards. Even on the training data we got less than 80% accuracy. However, we have only just begun building our architecture so this is not so bad at all! Certainly better than the average software developer diagnosing diabetes.

An important not is that this is just an evaluation of the training accuracy. While the authors suggest we could split this into a test and train dataset, a good practice is to further split into a cross-validation set. This will help us compare later models.

Note that accuracy is not the only metric available in Keras. The above line should print all of the configured metrics neatly though.

## Predictions

We can now use the model to make predictions with .predict() on some input. We can reuse our training set to see what the predictions were. Here we just print the first 10.

In [12]:
# calculate predictions
predictions = model.predict(X, verbose=0)
# round predictions
rounded = [round(x[0]) for x in predictions]
print(rounded[:10])

[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]
